In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from time import sleep
import pandas as pd
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
import requests

In [90]:
DRIVER_PATH = 'chromedriver'
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=options)

In [91]:
root_url = "https://www.sephora.com/"
driver.get(root_url)

In [92]:
click_title = WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.LINK_TEXT, "Makeup")))

In [93]:
action = ActionChains(driver)    
action.click(click_title).key_down(Keys.CONTROL).key_up(Keys.CONTROL).perform() 

In [94]:
sleep(10)

In [95]:
click_title.click()

In [96]:
current_url = driver.current_url
current_url

'https://www.sephora.com/shop/makeup-cosmetics'

In [98]:
def get_url(height):
    all_urls = set()
    
    # giving some time to load
    page = 1
    while True:
        product_url = f"{current_url}?currentPage={page}"
        driver.get(product_url)
        time.sleep(10)
        # Scroll down until no more content is loaded
        # scrolling once code        
        driver.execute_script(f"window.scrollTo(0, {height});")
        # giving time to load
        time.sleep(10) # wait for content to load

        elm_products = driver.find_elements(By.CSS_SELECTOR, "body > div:nth-child(3) > div > div > div > div.css-1t2x5d0.eanm77i0 > main > div:nth-child(3) > div.css-1322gsb > * > a")
        for product in elm_products:
            url = product.get_attribute("href")
            all_urls.add(url)
        
        height += 200
        print(len(all_urls))
        if len(all_urls) == 60*page:
            print(f"Loaded all urls from the page {page}")      
            page += 1       
            height = 300

        if page == 46:
            break
        
    return all_urls

In [ ]:
all_urls = get_url(300)

6
6
9
9
13
13
16
16
20
20
24
24
28
32
32
36
36
40
40
44
44
48
52
52
56
56
60
Loaded all urls from the page 1
66
66
69
69
73
73
76
76
80
80
84
88
88
92
92
96
100
100
104
104
108
108
112
116
116
120
Loaded all urls from the page 2
126
126
129
129
133
133
136
136
140
144
144
148
148
152
152
156
160
160
164
164
168
168
172
172
176
179
179
179
179
179
179
179
179
179
179


In [526]:
url = 'https://www.sephora.com/product/makeup-by-mario-soft-sculpt-shaping-stick-P472327?skuId=2457299&icid2=products%20grid:p472327:product'
driver.get(url)

In [527]:
dfcg = pd.DataFrame(columns=['url', 'product name', 'full content'])
dfcg.at[len(dfcg), 'url'] = url

In [528]:
name_product = driver.find_element(By.XPATH, "//span[contains(@data-at, 'product_name')]")

dfcg.at[len(dfcg)-1, 'product name'] = name_product.text

In [529]:
elm_contents = driver.find_elements(By.CSS_SELECTOR, "body > div:nth-child(3) > main > div:nth-child(1) > div.css-32uy52.eanm77i0 > div:nth-child(2) > div > div/*")

# contents = ''
# for content in elm_contents:
#     contents += content.get_attribute("textContent")
#     contents += "\n"
    
# dfcg.at[len(dfcg)-1, 'full content'] = contents

In [530]:
html_content = elm_contents[0].get_attribute("innerHTML")

In [542]:
soup_content = BeautifulSoup(html_content, 'html.parser')

In [543]:
soup_content

<b>What it is: </b>A dual-ended cream shaping stick that is buildable and swipes on seamlessly for true-to-skin definition that dries down to a natural matte finish.<br/><b>Ingredient Callouts:</b> It is cruelty-free.<br/><br/><b>What Else You Need to Know:</b> This dual-ended shaping stick combines Mario’s signature contour technique into one product that creates an effortless, softly sculpted look. The shades complement complexions regardless of undertone and are never orange or muddy. They also feature a removable angled buffing brush for effortless application.<br/><br/>This product is an Allure Best of Beauty award winner.

In [532]:
print(soup_content.get_text(separator='\n', strip = True))

What it is:
A dual-ended cream shaping stick that is buildable and swipes on seamlessly for true-to-skin definition that dries down to a natural matte finish.
Ingredient Callouts:
It is cruelty-free.
What Else You Need to Know:
This dual-ended shaping stick combines Mario’s signature contour technique into one product that creates an effortless, softly sculpted look. The shades complement complexions regardless of undertone and are never orange or muddy. They also feature a removable angled buffing brush for effortless application.
This product is an Allure Best of Beauty award winner.


In [539]:
for elem in soup_content.find_all(["a", "p", "div", "h3", "br"]):
    elem.replace_with(elem.text + "\n")

In [540]:
soup_content

<b>What it is: </b>A dual-ended cream shaping stick that is buildable and swipes on seamlessly for true-to-skin definition that dries down to a natural matte finish.
<b>Ingredient Callouts:</b> It is cruelty-free.

<b>What Else You Need to Know:</b> This dual-ended shaping stick combines Mario’s signature contour technique into one product that creates an effortless, softly sculpted look. The shades complement complexions regardless of undertone and are never orange or muddy. They also feature a removable angled buffing brush for effortless application.

This product is an Allure Best of Beauty award winner.